In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import json

In [28]:
df_chunk = pd.read_csv("outputs/icu_discharge_merged.csv.gz", chunksize=1000)
df_icu_discharge = next(df_chunk)
df_icu_discharge.head()

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los,note_id,note_type,note_seq,charttime,storetime,text
0,10000032,29079034,39553978,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266,10000032-DS-23,DS,23,2180-07-25 00:00:00,2180-07-25 21:42:00,\nName: ___ Unit No: _...
1,10000980,26913865,39765666,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2189-06-27 08:42:00,2189-06-27 20:38:27,0.497535,10000980-DS-21,DS,21,2189-07-03 00:00:00,2189-07-03 19:50:00,\nName: ___ Unit No: ___\n \nAdmi...
2,10001217,24597018,37067082,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-11-20 19:18:02,2157-11-21 22:08:00,1.118032,10001217-DS-4,DS,4,2157-11-25 00:00:00,2157-11-25 17:26:00,\nName: ___ Unit No: ___\n \n...
3,10001217,27703517,34592300,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-12-19 15:42:24,2157-12-20 14:27:41,0.948113,10001217-DS-5,DS,5,2157-12-24 00:00:00,2157-12-24 15:57:00,\nName: ___ Unit No: ___\n \n...
4,10001725,25563031,31205490,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2110-04-11 15:52:22,2110-04-12 23:59:56,1.338588,10001725-DS-12,DS,12,2110-04-14 00:00:00,2110-04-19 17:44:00,\nName: ___ Unit No: ___\n \nA...


In [10]:
test_subject_id = df_icu_discharge["hadm_id"].unique()[100]
print(test_subject_id)

25193617


In [12]:
df_example = df_icu_discharge[df_icu_discharge['hadm_id'] == test_subject_id]
discharge_summary = df_example.iloc[0, :]['text']

print(discharge_summary)

 
Name:  ___                     Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   M
 
Service: CARDIOTHORACIC
 
Allergies: 
lisinopril
 
Attending: ___.
 
Chief Complaint:
SOB/left sided CP
 
Major Surgical or Invasive Procedure:
___
CT guided placement of left pleural pigtail catheter

___ Bronchoscopy with attempted EBV placement

___
Left thoracoscopy, lysis of adhesions, resection of bleb, 
pleurodesis and creation of subcutaneous vent.

 
History of Present Illness:
Mr. ___ is a ___ y/o M with a h/o CAD, afib on xarelto,
pneumothorax approximately ___ years prior, chronic kidney
disease, NIDDM who presents as transfer from OSH after being
found to have a pneumothorax. 

Chest pain began approximately ___ although patient unsure. The
initial pain was associated with dyspnea and prompted him to
present to his primary care provider who referred him to ___. CXR on presentation there was notable for large left
pneumothora

### Load LLM

In [13]:
import os
import torch
import transformers
from transformers import BertTokenizer, BertModel
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig
import gc
#import flash_attention
#from flash_attention import rotary
#from flash_attn import rms_norm
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
# torch.cuda.empty_cache()

In [14]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-14B-Chat", trust_remote_code=True)

# use bf16
# model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-7B-Chat", device_map="auto", trust_remote_code=True, bf16=True).eval()
# use fp16
# model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-7B-Chat", device_map="auto", trust_remote_code=True, fp16=True).eval()
# use cpu only
# model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-7B-Chat", device_map="cpu", trust_remote_code=True).eval()
# use auto mode, automatically select precision based on the device.
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen-14B-Chat",
    device_map="cuda",
    trust_remote_code=True,
    bf16=True
).eval()

Try importing flash-attention for faster inference...


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

#### Extract Q&A pairs based on the discharge summary

In [15]:
# extract information from discharge summary
summary_text = discharge_summary
question_text = "Given the discharge summary above. extract discharge diagnosis from the given text, just give the answer briefly."
input_text = summary_text + " " + question_text

response, history = model.chat(tokenizer, query=input_text, history=None)
torch.cuda.empty_cache()
gc.collect()
print(response)

The discharge diagnosis is "Persistent left pneumothorax with subcutaneous emphysema" and "Urinary retention".


In [16]:
# extract information from discharge summary
summary_text = discharge_summary
question_text = "Given the discharge summary above. extract medication the patient should take from the given text, just give the medication name."
input_text = summary_text + " " + question_text

response, history = model.chat(tokenizer, query=input_text, history=None)
torch.cuda.empty_cache()
gc.collect()
print(response)

The patient should continue to take the following medications:

  1. Acetaminophen 1000 mg PO Q8H
  2. Atorvastatin 40 mg PO QPM
  3. Docusate Sodium 100 mg PO BID
  4. Lidocaine 5% Patch 1 PTCH TD QAM
  5. OxyCODONE ( Immediate Release) ___ mg PO Q6H PRN Pain - Moderate
  6. Allopurinol ___ mg PO DAILY
  7. GlipiZIDE 10 mg PO DAILY
  8. Levothyroxine Sodium 137 mcg PO DAILY
  9. Losartan Potassium 100 mg PO DAILY
  10. MetFORMIN (Glucophage) ___ mg PO DAILY
  11. Metoprolol Succinate XL 50 mg PO DAILY
  12. Omeprazole 20 mg PO BID
  13. Rivaroxaban 20 mg PO DAILY
  14. Tamsulosin 0.8 mg PO QHS
  15. Torsemide 5 mg PO DAILY
  16. Zolpidem Tartrate 10 mg PO QHS


In [17]:
# extract information from discharge summary
summary_text = discharge_summary
question_text = "Can you provide the discharge instructions for the patient according to the discharge summary?"
input_text = summary_text + " " + question_text

response, history = model.chat(tokenizer, query=input_text, history=None)
torch.cuda.empty_cache()
gc.collect()
print(response)

Discharge instructions for Mr. ___:

  * Continue using the incentive spirometer 10 times an hour while awake.
  * Check incisions daily and report any increased redness or drainage. Cover the area with a gauze pad if it is draining.
  * The chest tube dressing may be removed on ___ and the stitches will be removed by the ___.
  * The left anterior chest wound will close gradually from inside out (no stitches). Daily dressing changes will be necessary until the area is healed.
  * Going home with a bladder catheter in place which will remain in until you see Urology next week. It will keep your bladder decompressed and make the next voiding trial easier. The catheter is hooked up to a leg bag and you can empty that ___ times a day directly in the toilet by opening the spout.
  * May need pain medication once you are home but wean it over the next week as the discomfort resolves. Make sure to have regular bowel movements while on narcotic pain medication as they are constipating which c

#### Generate questions based on the templates

In [18]:
question_text = "What is the patient's chief complaint?"
prompt_text = "Change the grammatical form of the question and give 10 different ways of asking it."
input_text = question_text + prompt_text

response, history = model.chat(tokenizer, query=input_text, history=None)
torch.cuda.empty_cache()
gc.collect()
print(response)

Original question: What is the patient's chief complaint?

Rewritten questions:
1. What problem is the patient experiencing?
2. In what way is the patient feeling unwell?
3. Can you tell me what the patient's main concern is?
4. What symptom is causing the most distress for the patient?
5. What is the patient's primary complaint regarding their health?
6. How would you describe the patient's chief complaint?
7. What issue has the patient come to see you about today?
8. What is the matter that the patient wants to discuss with you?
9. Could you please tell me what the patient is troubled by?
10. What health concern has prompted the patient to seek medical attention?


#### Process the question files

In [19]:
def parse_questions(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = [line.strip() for line in f.readlines()]

    mother_questions = []
    child_questions = []
    current_mother_question = None
    current_child_list = []

    for line in lines:
        if line == "":  # space line indicates the end of a mother question and child list
            if current_mother_question is not None:
                child_questions.append(current_child_list)
                current_mother_question = None
                current_child_list = []
        else:
            if current_mother_question is None:
                # the first line is mother question
                current_mother_question = line
                mother_questions.append(current_mother_question)
            else:
                # other lines are child questions
                current_child_list.append(line)

    # process the last mother question and child list
    if current_mother_question is not None:
        child_questions.append(current_child_list)

    return mother_questions, child_questions

# example usage
file_path = "files/discharge_summary_questions.txt"
mother_questions, child_questions = parse_questions(file_path)

# print the results
print("question template list:", mother_questions)
print("question dict list", child_questions)

question template list: ['What are the allergies of the patient?', "What is the patient's chief complaint?", 'What is the major surgical or invasive procedure according to the discharge summary?', 'Describe the illness history according to the discharge summary.', 'What is the medical history of the patient?', 'What is the family history of the patient?', 'Describe the hospital course briefly.', 'What medication on admission is given to the patient?', 'What medication on discharge is given to the patient?', 'What is the discharge disposition of the patient?', 'List the discharge diagnosis of the patient.', 'What is the discharge condition of the patient?', 'What follow-up care was recommended after discharge?', 'What are the discharge instructions for the patient?', 'List the medication name and dosage on discharge of the patient. Each medication has one row.']
question dict list [['What allergies does the patient have?', 'What is the patient allergic to?', 'Can you list the patient’s 

In [21]:
print(mother_questions[0])
for i in enumerate(child_questions[0]):
    print(i)

What are the allergies of the patient?
(0, 'What allergies does the patient have?')
(1, 'What is the patient allergic to?')
(2, 'Can you list the patient’s allergies?')
(3, 'What substances is the patient sensitive to?')
(4, 'Does the patient have any known allergies?')
(5, 'What are the known allergens for the patient?')
(6, 'What allergic reactions has the patient experienced?')
(7, 'Are there any allergies that the patient is aware of?')
(8, 'What items trigger allergic responses in the patient?')
(9, 'Can you specify the patient’s allergic sensitivities?')


In [22]:
for index, row in df_icu_discharge.loc[:10, :].iterrows():
    subject_id = row['subject_id']
    hadm_id = row['hadm_id']
    # discharge_summary = row['text']
    
    # print hadm_id and subject_id
    print(f"hadm_id: {hadm_id}, subject_id: {subject_id}")

hadm_id: 29079034, subject_id: 10000032
hadm_id: 26913865, subject_id: 10000980
hadm_id: 24597018, subject_id: 10001217
hadm_id: 27703517, subject_id: 10001217
hadm_id: 25563031, subject_id: 10001725
hadm_id: 26184834, subject_id: 10001884
hadm_id: 23581541, subject_id: 10002013
hadm_id: 20345487, subject_id: 10002155
hadm_id: 23822395, subject_id: 10002155
hadm_id: 28994087, subject_id: 10002155
hadm_id: 22725460, subject_id: 10002348


#### Load and check the output json file

In [23]:
import json

# assign json file path
file_path = 'outputs/discharge_summary_qa/10000032_29079034.json'  # 替换为你的文件路径

# 加载 JSON 文件
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# 打印加载的数据
print(data)

# 访问具体字段
hadm_id = data['hadm_id']
subject_id = data['subject_id']
question_answer_pairs = data['question_answer_pairs']

print(f"HADM ID: {hadm_id}")
print(f"Subject ID: {subject_id}")
print("Question-Answer Pairs:")
for pair in question_answer_pairs:
    print(f"Question Template: {pair['question_template']}")
    print(f"Question: {pair['question']}")
    print(f"Answer: {pair['answer']}")
    print()  # 输出空行以增加可读性

{'hadm_id': 29079034, 'subject_id': 10000032, 'question_answer_pairs': [{'question_template': 'What are the allergies of the patient?', 'question': 'Does the patient have any known allergies?', 'answer': 'Yes, the patient has allergies to Percocet and Vicodin.'}, {'question_template': "What is the patient's chief complaint?", 'question': 'What is the key symptom the patient is presenting with?', 'answer': 'The key symptom that the patient is presenting with is altered mental status.'}, {'question_template': 'What is the major surgical or invasive procedure according to the discharge summary?', 'question': 'What does the discharge summary indicate as the major procedure?', 'answer': 'The discharge summary does not mention any major surgical or invasive procedures for the patient.'}, {'question_template': 'Describe the illness history according to the discharge summary.', 'question': 'How is the illness history detailed in the discharge summary?', 'answer': "The illness history in the di

### EHR QA generation

In [24]:
ehr_data_dir = "/home/mengliang/DatasetFolder/mimiciv/2.2"
ehr_patients_path = os.path.join(ehr_data_dir, "hosp/patients.csv.gz")
df_ehr_patients = pd.read_csv(ehr_patients_path, index_col=False)
df_ehr_patients.head()

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000032,F,52,2180,2014 - 2016,2180-09-09
1,10000048,F,23,2126,2008 - 2010,NaN
2,10000068,F,19,2160,2008 - 2010,NaN
3,10000084,M,72,2160,2017 - 2019,2161-02-13
4,10000102,F,27,2136,2008 - 2010,NaN


In [26]:
# give prompt and generate code 
prompt_text = "Given a dataframe, the first column is subject_id, a subject id means a patient, \
               the gender column means patient gender, M is male and F is female. the anchor_age column is age"
question_text = "generate Python code to query the number of patients who are older than 50 and whose gender is male."
input_text = prompt_text + " " + question_text

response, history = model.chat(tokenizer, query=input_text, history=None)
torch.cuda.empty_cache()
gc.collect()
print(response)

Assuming that the dataframe is called `df`, you can use the following code to query the number of patients who are older than 50 and whose gender is male:

```python
num_male_patients_older_than_50 = len(df[(df['gender'] == 'M') & (df['anchor_age'] > 50)])
```

Here's an explanation of what each part of the code does:

- `df['gender'] == 'M'` returns a boolean series indicating which rows have a male gender.
- `df['anchor_age'] > 50` returns a boolean series indicating which rows have an anchor age greater than 50.
- `(df['gender'] == 'M') & (df['anchor_age'] > 50)` returns a boolean series indicating which rows meet both conditions.
- `len(df[(df['gender'] == 'M') & (df['anchor_age'] > 50)])` returns the number of rows in the dataframe where both conditions are true.

Note that this assumes that there are no missing values in the gender or anchor_age columns of the dataframe. If there are missing values, you may need to handle them appropriately before running the query.


In [27]:
import pandas as pd

df = df_ehr_patients
# Query the number of patients who are older than 50 and whose gender is male
num_male_patients_older_than_50 = len(df[(df['gender'] == 'M') & (df['anchor_age'] > 50)])

print("Number of male patients older than 50:", num_male_patients_older_than_50)

Number of male patients older than 50: 67625
